<a href="https://colab.research.google.com/github/gowthaml15/Learning/blob/main/Resnet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [1]:
from torch import nn
from torchvision import datasets
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

### Dataset

In [9]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size = 0.1,
                shuffle = True,
                test = False):

  # Normalize the data
  normalize = transforms.Normalize(
      mean = [0.49,0.48,0.44],
      std = [0.20,0.19,0.20])

  # Transforms
  transform = transforms.Compose(
      [
          transforms.Resize((224,224)),
          transforms.ToTensor(),
          normalize
      ]
  )

  if test:
    dataset = datasets.CIFAR10(data_dir,
                               train = False,
                               download = True,
                               transform = transform)
    dataloader = DataLoader(dataset,
                            batch_size = batch_size,
                            shuffle = shuffle)
    return dataloader

  train_dataset = datasets.CIFAR10(data_dir,
                                   train = True,
                                   download = True,
                                   transform = transform)
  num_train = len(train_dataset)
  indices = list(range(num_train))
  split = int(np.floor(valid_size*num_train))

  if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

  train_idx, valid_idx = indices[split:], indices[:split]

  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  train_loader = DataLoader(train_dataset,
                            batch_size = batch_size,
                            sampler = train_sampler)
  valid_loader = DataLoader(train_dataset,
                            batch_size = batch_size,
                            sampler = valid_sampler)

  return train_loader, valid_loader

train_loader, valid_loader = data_loader(data_dir="/content/data/",batch_size = 32)

Files already downloaded and verified


In [10]:
test_loader = data_loader(data_dir = "/content/data/", batch_size = 32, test = True)

Files already downloaded and verified


### Resnet Archietecture

In [11]:
class block (nn.Module):
  def __init__(self, in_channels, out_channels, downsample=None, stride = 1):
    super(block,self).__init__()
    self.relu = nn.ReLU()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1)
    self.bn1  = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.downsample = downsample

  def forward(self, x):
    identity = x
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    if self.downsample is not None:
      identity = self.downsample(identity)
    x += identity
    x = self.relu(x)

    return x

In [2]:
class Resnet(nn.Module):
  def __init__(self, block, image_channels, layers, num_classes = 10):
    super(Resnet,self).__init__()
    self.in_channels = 64
    self.relu = nn.ReLU()

    self.conv1 = nn.Conv2d(image_channels, 64, kernel_size = 7, stride = 2, padding = 3)
    self.bn1 = nn.BatchNorm2d(64)
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)

    self.layer1 = self._make_layer(block,layers[0],out_channels = 64, stride=1)
    self.layer2 = self._make_layer(block,layers[1],out_channels = 128, stride=2)
    self.layer3 = self._make_layer(block,layers[2],out_channels = 256, stride=2)
    self.layer4 = self._make_layer(block,layers[3],out_channels = 512, stride=2)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.softmax = nn.Softmax(dim=1)
    self.fc = nn.Linear(512,num_classes)

  def forward(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc(x)
    return x

  def _make_layer(self,block,layer,out_channels,stride):
    downsample = None
    layers = []
    if stride !=1 or self.in_channels != out_channels:
      downsample = nn.Sequential(
          nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride),
          nn.BatchNorm2d(out_channels)
      )

    layers.append(block(self.in_channels,out_channels,downsample,stride))
    self.in_channels = out_channels
    for i in range(layer-1):
      layers.append(block(self.in_channels,out_channels))

    return nn.Sequential(*layers)


In [20]:
num_classes = 10
num_epochs = 10
batch_size = 32
learning_rate = 0.01

model = Resnet(block=block, image_channels=3, layers = [3,4,6,3],num_classes = num_classes)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
total_step = len(train_loader)
total_step

1407

In [21]:
import gc

for epoch in tqdm(range(num_epochs)):
  for batch_idx, (image, label) in enumerate(train_loader):
    image = image.to(device)
    label = label.to(device)

    output = model(image)
    loss = criterion(output, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    del image, label, output
    torch.cuda.empty_cache()
    gc.collect()

  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}"  )

  with torch.no_grad():
    correct = 0
    total = 0
    for image,label in valid_loader:
      image = image.to(device)
      label = label.to(device)

      output = model(image)
      _, predicted = torch.max(output.data, 1)

      total += label.size(0)
      correct += (predicted == label).sum().item()

      del image, label, output
      torch.cuda.empty_cache()
      gc.collect()

    print(f"Accuracy of the network on the 10000 test images: {100 * correct // total} %")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.1334


 10%|█         | 1/10 [09:52<1:28:52, 592.50s/it]

Accuracy of the network on the 10000 test images: 44 %


Exception ignored in: <function _xla_gc_callback at 0x789c86ab96c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
 10%|█         | 1/10 [17:29<2:37:28, 1049.85s/it]


KeyboardInterrupt: ignored

In [15]:
#Testing results
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

Accuracy of the network on the 10000 test images: 85.41 %


### Pre-trained model

### Rough Works

In [ ]:
if 1!=1 or 64!=64*4:
  print("true")

true


In [ ]:
256*2

512

In [ ]:
m = nn.AdaptiveAvgPool2d((5, 7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
output.shape

torch.Size([1, 64, 5, 7])

In [ ]:
for idx, (data,label) in enumerate(train_loader):
  print(label.size(0))
  break

32


In [19]:
Resnet(block,3,[3,4,6,3],10)

Resnet(
  (relu): ReLU()
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): block(
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): block(
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Ba